## **Setting up Development Environment**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
% cd /content/drive/MyDrive/Colab Notebooks/Hate-Speech-Detection-in-Tamil/HASOC Tamil/Task 1

/content/drive/MyDrive/Colab Notebooks/Hate-Speech-Detection-in-Tamil/HASOC Tamil/Task 1


In [3]:
! pip install -U -q transformers
! pip install -U -q sentencepiece

     |████████████████████████████████| 3.4 MB 21.7 MB/s 
     |████████████████████████████████| 3.3 MB 48.9 MB/s 
     |████████████████████████████████| 596 kB 53.6 MB/s 
     |████████████████████████████████| 67 kB 5.3 MB/s 
     |████████████████████████████████| 895 kB 59.5 MB/s 
     |████████████████████████████████| 1.2 MB 27.4 MB/s 


## **Importing Libraries**

In [4]:
import pandas as pd

import numpy as np

from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

from transformers import AutoTokenizer, AutoModel

import pickle

## **Loading Data**

In [5]:
dataframe = pd.read_csv("Dataset/tamil_offensive_train.tsv", sep='\t')

In [6]:
dataframe.head()

,id,text,category
0,tam1,திருமலை நாயக்கர் பேரவை சார்பாக படம் வெற்றி பெற...,NOT
1,tam2,இந்த ட்ரெய்லர் கூட பார்க்கிற மாதிரி இல்லை.. இத...,OFF
2,tam3,மைசூரு செட்டியார் சமூகத்தின் சார்பாக இப்படம் வ...,NOT
3,tam4,மொத்த சாதியும் ஒரு சாதிக்கு எதிரா நிக்குது.......,OFF
4,tam5,only for விஜய் சேதுபதி and STR,NOT


In [7]:
dataframe.describe()

,id,text,category
count,5880,5880,5880
unique,5880,5812,3
top,tam54,வந்தா ராஜாவா தான் வருவேன்........ ராஜாவா வந்தா...,NOT
freq,1,5,4724


In [8]:
dataframe['category'].unique()

array(['NOT', 'OFF', 'not-Tamil'], dtype=object)

In [9]:
dataframe.shape

(5880, 3)

In [10]:
dataframe = dataframe.loc[dataframe['category'] != 'not-Tamil']
dataframe['category'].unique()

array(['NOT', 'OFF'], dtype=object)

In [11]:
dataframe.shape

(5877, 3)

In [12]:
text = dataframe['text']

In [13]:
text

0       திருமலை நாயக்கர் பேரவை சார்பாக படம் வெற்றி பெற...
1       இந்த ட்ரெய்லர் கூட பார்க்கிற மாதிரி இல்லை.. இத...
2       மைசூரு செட்டியார் சமூகத்தின் சார்பாக இப்படம் வ...
3       மொத்த சாதியும் ஒரு சாதிக்கு எதிரா நிக்குது.......
4                          only for விஜய் சேதுபதி and STR
                              ...                        
5875    இப்போதான் ஜாதி பிரச்சினை இல்லாம இருக்கு இப்போ ...
5876    இது ஒரு சாதி ரீதியான படம் இல்லை தான் செல்லும் ...
5877    கதை தெரியவில்லை  இருந்தாலும் மாறுபட்ட சிந்தனை ...
5878    அட பரவால்லையே 8.5 டிஸ்லைக்ஸ் போட்ருக்கானுங்க.....
5879    உதயநிதிக்கு நடிப்பு வரலனு சொல்ரவங்க லைக் போடுங...
Name: text, Length: 5877, dtype: object

In [14]:
label = dataframe['category']

In [15]:
label

0       NOT
1       OFF
2       NOT
3       OFF
4       NOT
       ... 
5875    NOT
5876    NOT
5877    NOT
5878    OFF
5879    NOT
Name: category, Length: 5877, dtype: object

## **Data Preprocessing**

In [16]:
le = LabelEncoder()
label = le.fit_transform(label)
label

array([0, 1, 0, ..., 0, 1, 0])

In [17]:
text = text.str.replace(r"[+/#@&*$%:]",'')
text = text.to_numpy()
text

array(['திருமலை நாயக்கர் பேரவை சார்பாக படம் வெற்றி பெற வாழ்த்துக்கள்',
       'இந்த ட்ரெய்லர் கூட பார்க்கிற மாதிரி இல்லை.. இதை தியேட்டர் ல போய் பார்க்கனுமா.. .',
       'மைசூரு செட்டியார் சமூகத்தின் சார்பாக இப்படம் வெற்றிபெற வாழ்த்துகிறேன்',
       ...,
       'கதை தெரியவில்லை  இருந்தாலும் மாறுபட்ட சிந்தனை வாழ்த்துக்கள்  பறையர் சார்பாக  நன்றி மோகன் ஜி',
       'அட பரவால்லையே 8.5 டிஸ்லைக்ஸ் போட்ருக்கானுங்க.. பரவால வயிறு எறிஞ்சி சாவுங்கடா சாவுங்கடா சாக்காடை நாய்ங்களா..இன்னும் நிறைய டிஸ்லைக்ஸ் எதிர் பாக்குறன்..',
       'உதயநிதிக்கு நடிப்பு வரலனு சொல்ரவங்க லைக் போடுங்கள் ...'],
      dtype=object)

## **Feature Extraction**

In [18]:
# Load Transformer Model

tokenizer = AutoTokenizer.from_pretrained("google/muril-base-cased")
model = AutoModel.from_pretrained("google/muril-base-cased")

Downloading:   0%|          | 0.00/181 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/411 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.02M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/113 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/909M [00:00<?, ?B/s]

Some weights of the model checkpoint at google/muril-base-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [19]:
# Sample Output

tokenized_input = tokenizer(
        text[0],
        padding=True,
        truncation=False,
        return_tensors='pt'
    )

sample_output = model(**tokenized_input)

In [20]:
print(tokenized_input)

{'input_ids': tensor([[   104,  78708,  40896,  56772,  26847,   7772,   6396,  14801, 120783,
            105]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


In [21]:
print(sample_output)

BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[-0.0017,  0.0020, -0.0001,  ..., -0.0105, -0.0019, -0.0028],
         [ 0.0035, -0.0008,  0.0005,  ..., -0.0105,  0.0034, -0.0010],
         [ 0.0083,  0.0032,  0.0007,  ..., -0.0173, -0.0008,  0.0021],
         ...,
         [-0.0181, -0.0026, -0.0022,  ..., -0.0119, -0.0029, -0.0044],
         [-0.0227,  0.0035, -0.0068,  ..., -0.0017, -0.0012, -0.0054],
         [-0.0017,  0.0020, -0.0001,  ..., -0.0105, -0.0019, -0.0028]]],
       grad_fn=<NativeLayerNormBackward0>), pooler_output=tensor([[ 9.5366e-03,  1.9333e-02, -7.6383e-03,  1.5530e-02, -7.0882e-03,
         -8.1754e-03, -1.8386e-03, -4.0856e-03, -3.2257e-03, -9.4593e-03,
         -9.5955e-03,  5.1454e-03,  1.6289e-03,  6.9413e-05, -4.6508e-03,
          1.7214e-02,  1.9794e-03, -8.6327e-03,  2.0441e-04, -5.8779e-03,
          5.9865e-03,  4.6209e-04,  6.6394e-03, -2.5243e-02,  5.0351e-03,
          1.3785e-03, -7.6058e-03, -6.0471e-03,  7.7698e-04, -2.318

In [22]:
print(sample_output.pooler_output.cpu().detach().numpy().shape)

(1, 768)


In [23]:
# Tokenizing Input Data

input = []

for i in text:
    tokenized_input = tokenizer(
        i,
        padding=True,
        truncation=False,
        return_tensors='pt'
    )
    input.append(tokenized_input)

In [25]:
# Embedding the Input Data

output = []

j = 1

for i in range(len(input)):
    
    model_output = model(**input[i])
    model_output = model_output.pooler_output.cpu().detach().numpy()
    output.append(model_output)
    
    # Saving Embedded Input Data to Disk

    if ((i + 1) % 200) == 0:
        
        file_name = "./MuRIL/MuRIL-No-Stem-Embedded-TrainData/output" + str(j) + ".pickle"
        
        with open(file_name, "wb") as fp:   
            # Pickling
            pickle.dump(output, fp)
        
        print(file_name + " done")
        output = []
        j += 1
        
file_name = "./MuRIL/MuRIL-No-Stem-Embedded-TrainData/output" + str(j) + ".pickle"

with open(file_name, "wb") as fp:   
    # Pickling
    pickle.dump(output, fp)

print(file_name + " done")
output = []

./MuRIL/MuRIL-No-Stem-Embedded-TrainData/output1.pickle done
./MuRIL/MuRIL-No-Stem-Embedded-TrainData/output2.pickle done
./MuRIL/MuRIL-No-Stem-Embedded-TrainData/output3.pickle done
./MuRIL/MuRIL-No-Stem-Embedded-TrainData/output4.pickle done
./MuRIL/MuRIL-No-Stem-Embedded-TrainData/output5.pickle done
./MuRIL/MuRIL-No-Stem-Embedded-TrainData/output6.pickle done
./MuRIL/MuRIL-No-Stem-Embedded-TrainData/output7.pickle done
./MuRIL/MuRIL-No-Stem-Embedded-TrainData/output8.pickle done
./MuRIL/MuRIL-No-Stem-Embedded-TrainData/output9.pickle done
./MuRIL/MuRIL-No-Stem-Embedded-TrainData/output10.pickle done
./MuRIL/MuRIL-No-Stem-Embedded-TrainData/output11.pickle done
./MuRIL/MuRIL-No-Stem-Embedded-TrainData/output12.pickle done
./MuRIL/MuRIL-No-Stem-Embedded-TrainData/output13.pickle done
./MuRIL/MuRIL-No-Stem-Embedded-TrainData/output14.pickle done
./MuRIL/MuRIL-No-Stem-Embedded-TrainData/output15.pickle done
./MuRIL/MuRIL-No-Stem-Embedded-TrainData/output16.pickle done
./MuRIL/MuRIL-No-

In [26]:
# Loading Embedded Input Data from Disk

output = []

for i in range(30):
    file_name = "./MuRIL/MuRIL-No-Stem-Embedded-TrainData/output" + str(i + 1) + ".pickle"
    with open(file_name, "rb") as fp:   #Pickling
        file_output = pickle.load(fp)
        for x in file_output:
            output.append(x)
    print(file_name + " done")

X = output
output = []

./MuRIL/MuRIL-No-Stem-Embedded-TrainData/output1.pickle done
./MuRIL/MuRIL-No-Stem-Embedded-TrainData/output2.pickle done
./MuRIL/MuRIL-No-Stem-Embedded-TrainData/output3.pickle done
./MuRIL/MuRIL-No-Stem-Embedded-TrainData/output4.pickle done
./MuRIL/MuRIL-No-Stem-Embedded-TrainData/output5.pickle done
./MuRIL/MuRIL-No-Stem-Embedded-TrainData/output6.pickle done
./MuRIL/MuRIL-No-Stem-Embedded-TrainData/output7.pickle done
./MuRIL/MuRIL-No-Stem-Embedded-TrainData/output8.pickle done
./MuRIL/MuRIL-No-Stem-Embedded-TrainData/output9.pickle done
./MuRIL/MuRIL-No-Stem-Embedded-TrainData/output10.pickle done
./MuRIL/MuRIL-No-Stem-Embedded-TrainData/output11.pickle done
./MuRIL/MuRIL-No-Stem-Embedded-TrainData/output12.pickle done
./MuRIL/MuRIL-No-Stem-Embedded-TrainData/output13.pickle done
./MuRIL/MuRIL-No-Stem-Embedded-TrainData/output14.pickle done
./MuRIL/MuRIL-No-Stem-Embedded-TrainData/output15.pickle done
./MuRIL/MuRIL-No-Stem-Embedded-TrainData/output16.pickle done
./MuRIL/MuRIL-No-

In [27]:
X = np.array(X)
X.shape

(5877, 1, 768)

In [28]:
X = X.reshape(5877, 768)
X.shape

(5877, 768)

In [29]:
y = label
y.shape

(5877,)

## **Train Test Split**

In [30]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [31]:
X_train.shape

(4701, 768)

In [32]:
X_val.shape

(1176, 768)

## **Logistic Regression**

In [33]:
lr = LogisticRegression()
lr.fit(X_train, y_train)
lr_pred = lr.predict(X_val)
print(classification_report(y_val, lr_pred))

              precision    recall  f1-score   support

           0       0.81      1.00      0.89       950
           1       0.00      0.00      0.00       226

    accuracy                           0.81      1176
   macro avg       0.40      0.50      0.45      1176
weighted avg       0.65      0.81      0.72      1176



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## **Support Vector Machine**

In [34]:
svm = SVC()
svm.fit(X_train, y_train)
svm_pred = svm.predict(X_val)
print(classification_report(y_val, svm_pred))

              precision    recall  f1-score   support

           0       0.81      1.00      0.89       950
           1       0.00      0.00      0.00       226

    accuracy                           0.81      1176
   macro avg       0.40      0.50      0.45      1176
weighted avg       0.65      0.81      0.72      1176



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## **Naive Bayes**

In [35]:
nb = GaussianNB()
nb.fit(X_train, y_train)
nb_pred = nb.predict(X_val)
print(classification_report(y_val, nb_pred))

              precision    recall  f1-score   support

           0       0.93      0.74      0.82       950
           1       0.41      0.75      0.53       226

    accuracy                           0.74      1176
   macro avg       0.67      0.75      0.68      1176
weighted avg       0.83      0.74      0.77      1176



## **Stochastic Gradient Descent**

In [36]:
sgd = SGDClassifier()
sgd.fit(X_train, y_train)
sgd_pred = sgd.predict(X_val)
print(classification_report(y_val, sgd_pred))

              precision    recall  f1-score   support

           0       0.81      1.00      0.89       950
           1       0.00      0.00      0.00       226

    accuracy                           0.81      1176
   macro avg       0.40      0.50      0.45      1176
weighted avg       0.65      0.81      0.72      1176



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## **K Nearest Neighbours**

In [37]:
knn = KNeighborsClassifier()
knn.fit(X_train, y_train)
knn_pred = knn.predict(X_val)
print(classification_report(y_val, knn_pred))

              precision    recall  f1-score   support

           0       0.84      0.95      0.89       950
           1       0.54      0.23      0.33       226

    accuracy                           0.81      1176
   macro avg       0.69      0.59      0.61      1176
weighted avg       0.78      0.81      0.78      1176



## **Decision tree**

In [38]:
dt = DecisionTreeClassifier()
dt.fit(X_train, y_train)
dt_pred = dt.predict(X_val)
print(classification_report(y_val, dt_pred))

              precision    recall  f1-score   support

           0       0.85      0.84      0.85       950
           1       0.38      0.40      0.39       226

    accuracy                           0.76      1176
   macro avg       0.61      0.62      0.62      1176
weighted avg       0.76      0.76      0.76      1176



## **Random Forest**

In [39]:
rf = RandomForestClassifier()
rf.fit(X_train, y_train)
rf_pred = rf.predict(X_val)
print(classification_report(y_val, rf_pred))

              precision    recall  f1-score   support

           0       0.84      0.98      0.91       950
           1       0.72      0.23      0.34       226

    accuracy                           0.83      1176
   macro avg       0.78      0.60      0.62      1176
weighted avg       0.82      0.83      0.80      1176



## **Majority Voting**

In [40]:
mv_pred = []

for i in range(len(lr_pred)):
    one = 0
    zero = 0
    # Keeping only KNN, RF & DT in predictions due to poor performance
    predictions = [rf_pred[i], knn_pred[i], dt_pred[i]]
    for pred in predictions:
        if pred == 1: one += 1
        if pred == 0: zero +=1
    if one > zero: mv_pred.append(1)
    else: mv_pred.append(0)

mv_pred = np.array(mv_pred)

print(classification_report(y_val, mv_pred))    

              precision    recall  f1-score   support

           0       0.84      0.97      0.90       950
           1       0.63      0.21      0.31       226

    accuracy                           0.82      1176
   macro avg       0.73      0.59      0.61      1176
weighted avg       0.80      0.82      0.79      1176



## **Loading Testing Data**

In [41]:
dataframe_test = pd.read_csv(
    "Dataset/tam_offesive_withoutlabels_test.tsv", 
    sep='\t'
)
dataframe_test.head()

,id,text
0,tam_test_1,தலைவா STR இதுக்குதான் கதுருந்தோம் மாஸ் தலைவா t...
1,tam_test_2,நாயுடு மக்கள் சார்பாக திரைப்படம் வெற்றி பெற வா...
2,tam_test_3,தில்லானா முயற்சி தஞ்சை கோனார்
3,tam_test_4,திரௌபதியின் துகிலுறித்த போது காத்த கண்ணனின் யா...
4,tam_test_5,நான் தியேட்டர்லே படம் பார்த்து 35 வருஷமாச்சு! ...


In [42]:
dataframe_test.describe()

,id,text
count,654,654
unique,654,654
top,tam_test_646,வீர சைவ குலத்தின் சார்பாக படம் வெற்றி பெற வாழ்...
freq,1,1


In [43]:
text_test = dataframe_test['text']

## **Test Data Preprocessing**

In [44]:
text_test = text_test.str.replace(r"[+/#@&*$%:]",'')
text_test = text_test.to_numpy()

## **Test Feature Extraction**

In [45]:
# Load Transformer Model

tokenizer = AutoTokenizer.from_pretrained("google/muril-base-cased")
model = AutoModel.from_pretrained("google/muril-base-cased")

Some weights of the model checkpoint at google/muril-base-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [46]:
# Sample Output

tokenized_input = tokenizer(
        text_test[0],
        padding=True,
        truncation=False,
        return_tensors='pt'
    )

sample_output = model(**tokenized_input)

In [47]:
print(tokenized_input)

{'input_ids': tensor([[   104,  20005,   7848,  39159,   4095,   1394,   1895,   4365,    806,
           1844,  40017,  14088,  25557,   2131,  20005,   7848,  34654,   9278,
          92683,   2921, 167797,    105]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


In [48]:
print(sample_output)

BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[-2.4294e-03,  1.4259e-03,  5.6737e-04,  ..., -1.0324e-02,
          -2.6184e-03, -2.3275e-03],
         [ 3.7599e-04,  5.6072e-03, -1.7263e-03,  ..., -1.5888e-02,
           2.6076e-03,  1.0137e-03],
         [ 1.1785e-02,  4.8415e-03,  3.8222e-03,  ..., -1.3923e-02,
          -9.2336e-04, -2.4995e-03],
         ...,
         [-1.1422e-03, -1.7787e-03, -6.4589e-05,  ..., -8.1838e-03,
           1.3760e-03, -3.7553e-03],
         [-3.6026e-03, -1.3796e-03, -1.6834e-03,  ..., -1.6792e-02,
          -1.4780e-03, -2.3770e-03],
         [-2.4294e-03,  1.4259e-03,  5.6737e-04,  ..., -1.0324e-02,
          -2.6184e-03, -2.3275e-03]]], grad_fn=<NativeLayerNormBackward0>), pooler_output=tensor([[ 9.2824e-03,  1.8399e-02, -8.4186e-03,  1.6888e-02, -9.1333e-03,
         -8.3700e-03, -1.8257e-03, -3.2925e-03, -3.2882e-03, -1.0341e-02,
         -1.0240e-02,  5.8044e-03,  2.1776e-03,  1.2503e-03, -3.7515e-03,
          1.8307e-

In [49]:
print(sample_output.pooler_output.cpu().detach().numpy().shape)

(1, 768)


In [50]:
# Tokenizing Test Input Data

input = []

for i in text_test:
    tokenized_input = tokenizer(
        i,
        padding=True,
        truncation=False,
        return_tensors='pt'
    )
    input.append(tokenized_input)

In [51]:
# Embedding the Test Input Data

output = []

j = 1

for i in range(len(input)):
    
    model_output = model(**input[i])
    model_output = model_output.pooler_output.cpu().detach().numpy()
    output.append(model_output)
    
    # Saving Embedded Test Input Data to Disk

    if ((i + 1) % 200) == 0:
        
        file_name = "./MuRIL/MuRIL-No-Stem-Embedded-TestData/output" + str(j) + ".pickle"
        
        with open(file_name, "wb") as fp:   
            # Pickling
            pickle.dump(output, fp)
        
        print(file_name + " done")
        output = []
        j += 1
        
file_name = "./MuRIL/MuRIL-No-Stem-Embedded-TestData/output" + str(j) + ".pickle"

with open(file_name, "wb") as fp:   
    # Pickling
    pickle.dump(output, fp)

print(file_name + " done")
output = []

./MuRIL/MuRIL-No-Stem-Embedded-TestData/output1.pickle done
./MuRIL/MuRIL-No-Stem-Embedded-TestData/output2.pickle done
./MuRIL/MuRIL-No-Stem-Embedded-TestData/output3.pickle done
./MuRIL/MuRIL-No-Stem-Embedded-TestData/output4.pickle done


In [52]:
# Loading Embedded Input Test Data from Disk

output = []

for i in range(4):
    file_name = "./MuRIL/MuRIL-No-Stem-Embedded-TestData/output" + str(i + 1) + ".pickle"
    with open(file_name, "rb") as fp:   #Pickling
        file_output = pickle.load(fp)
        for x in file_output:
            output.append(x)
    print(file_name + " done")

X_test = output
output = []

./MuRIL/MuRIL-No-Stem-Embedded-TestData/output1.pickle done
./MuRIL/MuRIL-No-Stem-Embedded-TestData/output2.pickle done
./MuRIL/MuRIL-No-Stem-Embedded-TestData/output3.pickle done
./MuRIL/MuRIL-No-Stem-Embedded-TestData/output4.pickle done


In [53]:
X_test = np.array(X_test)
X_test.shape

(654, 1, 768)

In [54]:
X_test = X_test.reshape(654, 768)
X_test.shape

(654, 768)

## **Getting Predictions**

In [55]:
# Logistic Regression
lr_test_pred = lr.predict(X_test)

# Support Vector Machine
svm_test_pred = svm.predict(X_test)

# Naive Bayes
nb_test_pred = nb.predict(X_test)

# Stochastic Gradient Descent
sgd_test_pred = sgd.predict(X_test)

# K Nearest Neighbours
knn_test_pred = knn.predict(X_test)

# Decision Tree
dt_test_pred = dt.predict(X_test)

# Random Forest
rf_test_pred = rf.predict(X_test)

In [56]:
# Majority Voting
mv_test_pred = []

for i in range(len(lr_test_pred)):
    one = 0
    zero = 0
    predictions = [rf_test_pred[i], knn_test_pred[i], dt_test_pred[i]]
    for pred in predictions:
        if pred == 1: one += 1
        if pred == 0: zero +=1
    if one > zero: mv_test_pred.append(1)
    else: mv_test_pred.append(0)

mv_test_pred = np.array(mv_test_pred)

## **Evaluating the Predictions**

In [57]:
dataframe_eval = pd.read_csv(
    "Dataset/tamil_offensive_test task 1 with labels.tsv", 
    sep='\t'
)

In [58]:
y_test = dataframe_eval['category']
y_test = LabelEncoder().fit_transform(y_test)

In [59]:
print('Logistic Regression')
print(classification_report(y_test, lr_test_pred))

Logistic Regression
              precision    recall  f1-score   support

           0       0.82      1.00      0.90       536
           1       0.00      0.00      0.00       118

    accuracy                           0.82       654
   macro avg       0.41      0.50      0.45       654
weighted avg       0.67      0.82      0.74       654



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [60]:
print('Support Vector Machine')
print(classification_report(y_test, svm_test_pred))

Support Vector Machine
              precision    recall  f1-score   support

           0       0.82      1.00      0.90       536
           1       0.00      0.00      0.00       118

    accuracy                           0.82       654
   macro avg       0.41      0.50      0.45       654
weighted avg       0.67      0.82      0.74       654



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [61]:
print('Naive Bayes')
print(classification_report(y_test, nb_test_pred))

Naive Bayes
              precision    recall  f1-score   support

           0       0.94      0.77      0.85       536
           1       0.42      0.76      0.55       118

    accuracy                           0.77       654
   macro avg       0.68      0.77      0.70       654
weighted avg       0.84      0.77      0.79       654



In [62]:
print('Stochastic Gradient Descent')
print(classification_report(y_test, sgd_test_pred))

Stochastic Gradient Descent
              precision    recall  f1-score   support

           0       0.82      1.00      0.90       536
           1       0.00      0.00      0.00       118

    accuracy                           0.82       654
   macro avg       0.41      0.50      0.45       654
weighted avg       0.67      0.82      0.74       654



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [63]:
print('K Nearest Neighbours')
print(classification_report(y_test, knn_test_pred))

K Nearest Neighbours
              precision    recall  f1-score   support

           0       0.85      0.96      0.90       536
           1       0.59      0.25      0.36       118

    accuracy                           0.83       654
   macro avg       0.72      0.61      0.63       654
weighted avg       0.81      0.83      0.81       654



In [64]:
print('Decision Tree')
print(classification_report(y_test, dt_test_pred))

Decision Tree
              precision    recall  f1-score   support

           0       0.86      0.86      0.86       536
           1       0.38      0.39      0.38       118

    accuracy                           0.78       654
   macro avg       0.62      0.62      0.62       654
weighted avg       0.78      0.78      0.78       654



In [65]:
print('Random Forest')
print(classification_report(y_test, rf_test_pred))

Random Forest
              precision    recall  f1-score   support

           0       0.85      0.97      0.90       536
           1       0.60      0.21      0.31       118

    accuracy                           0.83       654
   macro avg       0.72      0.59      0.61       654
weighted avg       0.80      0.83      0.80       654



In [66]:
print('Majority Voting')
print(classification_report(y_test, mv_test_pred))

Majority Voting
              precision    recall  f1-score   support

           0       0.89      0.90      0.90       536
           1       0.52      0.49      0.51       118

    accuracy                           0.83       654
   macro avg       0.71      0.70      0.70       654
weighted avg       0.82      0.83      0.83       654

